<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/RelativeStrengthIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

삼성전자의 RSI(RelativeStrengthIndex)를 구해본다.<BR>
먼저 필요한 모듈을 임포트 한다.

In [20]:
import yfinance as yf
import pandas as pd
import numpy as np

삼성전자의 2022년 하반기 주가 중 조정종가를 읽어와서 dataframe에 넣는다.

In [49]:
df = yf.download( "005930.KS" , "2022-07-01", "2022-12-31", progress=False)['Adj Close']

주가흐름을 그려본다.

In [ ]:
df.plot()

직전일과 오늘의 가격의 차이를 구한 다음 배열에 저장한다.

In [51]:
price_changes = np.diff( df )

가격차이 중 가격이 오른날은 gain에, 가격이 내린날은 loss에 저장한다.<BR>
gain은 가격이 오른날은 오른 가격을, 나머지 날은 모두 0으로 채운다. <BR>
loss는 가격이 내린날은 내린 가격을, 나머지 날은 모두 0으로 채운다.

In [52]:
gain = np.where( price_changes > 0, price_changes, 0)
loss = np.where( price_changes < 0,  -price_changes , 0)

RSI는 통상 14일 간의 상승 하락을 본다. 여기서도 기간을 14일로 설정한다.

In [53]:
period = 14

1년 간의 RSI 값 변화를 저장할 배열을 하나 선언한다.

In [54]:
rsi = np.zeros( len(price_changes) - period )

루프를 돌며, 14일의 윈도우를 슬라이딩 하며, 1년간의 RSI 변화를 계산하고, 이를 배열에 담는다.

In [55]:
for i in range(0, len(price_changes) - period ):
  avg_gain = np.mean(gain[i:period+i])
  avg_loss = np.mean(loss[i:period+i])
  rsi[i] = 100 - (100 / (1 + (avg_gain / avg_loss)))

그래프로 RSI 변화를 도식화 해 본다.

In [ ]:
import matplotlib.pyplot as plt
x = df.index[:-period-1] # x축에는 날짜를 기록한다.
fig, ax1 = plt.subplots()

# step1:  x축과 왼쪽 Y축 레이블을 설정하고, 색깔을 지정한다.
ax1.set_xlabel('DATE')
ax1.set_ylabel("RSI", color='b')

# step2:  왼쪽 축에 RSI 그래프를 그린다.
ax1.plot(x, rsi, color= 'b')

# step3:  축 글씨도 파란색으로 표시한다.
ax1.tick_params(axis='y', labelcolor='b')

# RSI 강도가 70이 넘는 부분에 그린으로 음영을 표시한다.
ax1.axhspan(70, 100, color='green', alpha=0.3)

# RSI 강도가 430 이하인 부분에 대해 회색으로 음영을 표시한다.
ax1.axhspan(0, 30, color='gray', alpha=0.3)

# step4: 이제 오른쪽 축을 생성한다.
ax2 = ax1.twinx()

# step5: 오른쪽 y축에 표시할 레이블을 지정한다.
ax2.set_ylabel("Samsung Prices", color='r')

# step6: 오른쪽 축에 삼성전자 주가를 그린다.
ax2.plot(x, df[:-period-1], color='r')

# step7: 오른쪽 축 글씨도 붉은색으로 한다.
ax2.tick_params(axis='y', labelcolor='r')

# step8: 여백을 자동설정하고, 그래프를 그린다.
fig.tight_layout()
plt.show()